# BTC Solar Solver


In [ ]:
# Custom Modules
import SolarSimulator


# House Info
pv_system_info = {'location':{
		'lat':33.392,
		'lon':-111.921},
	'specs':{
		'array_type':1,
		'azimuth':180,
		'tilt':28.7,
		'losses':1}}

In [3]:
import os
from datetime import datetime
import pandas as pd
import numpy as np
import PySAM.Pvwattsv7 as Pvwatts
import requests

# Latitude and longitude of location
latitude = 40.71
longitude = -74.01

# NSRDB API key
nsrdb_api_key = 'bfpWE6iufkhWFk9DfN1G7M4FYeUIlWNp3MCOaQ2Z'

# System specifications
system_capacity = 10  # kW
module = 'Canadian_Solar_CS5P_220M___2009_'
inverter = 'SMA_America__SC800CP_US__240V_'

# Dates for simulation
start_date = pd.Timestamp('2022-01-01', tz='America/New_York')
end_date = pd.Timestamp('2022-12-31', tz='America/New_York')
times = pd.date_range(start=start_date, end=end_date, closed='left', freq='H')

# Fetch weather data from NSRDB API
nsrdb_url = f'https://developer.nrel.gov/api/solar/nsrdb_v2.json?api_key={nsrdb_api_key}&lat={latitude}&lon={longitude}&attributes=dhi,dni,ghi,temp_air,wspd'
response = requests.get(nsrdb_url)
data = response.json()['outputs']['hourly']
weather = pd.DataFrame(data)
weather.index = pd.to_datetime(weather['timestamp'], utc=True).dt.tz_convert('America/New_York')

# PySAM model object
model = Pvwatts.default("Pvwattsv7")
model.SolarResource.solar_resource_data = weather[['dhi', 'dni', 'ghi', 'temp_air', 'wspd']]
model.SolarResource.tilt = 0
model.SolarResource.azimuth = 180
model.SolarResource.solar_resource_model_choice = 0
model.SolarResource.system_capacity = system_capacity
model.Module.module_model = module
model.Inverter.inverter_model = inverter
model.Inverter.mppt_low_inverter = 0
model.Inverter.mppt_high_inverter = system_capacity
model.Inverter.inverter_efficiency = 0.96
model.execute()

ModuleNotFoundError: No module named 'PySAM'